### Steps:
1. Run cell 2, 3 (Imports)
2. Update cell 4 according to your Test (retailer, year, period).
3. Run your selected Test

In [0]:
import os
import re
from datetime import datetime
from typing import Any, Optional, Union

import pytz
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F

MARS_CALENDAR = "raw_manual_us.mars_calendar"
DIM_CALENDAR_DATE = "cdm_common.calendar_date"
DIM_CALENDAR_PERIOD = "cdm_common.calendar_period"
NIELSEN_PRODUCT = "dcom_gsc_silver.us_nielsen_product"
CUSTOMER_DIC = {
    "AHOLD": "US_USMWCONFTOTAL_AHOLDDELHAIZETOTALSTOREDELIVERYTA",
    "ALBSCO": "US_USMWCONFTOTAL_ALBSCOTOTALECOMTA",
    "GIANTEAGLE": "US_USMWCONFTOTAL_GIANTEAGLETOTALECOMTA",
    "HYVEE": "US_USMWCONFTOTAL_HYVEETOTALECOMTA",
    "LOWESFOOD": "US_USMWCONFTOTAL_LOWESFOODTOTALECOMTA",
    "MEIJER": "US_USMWCONFTOTAL_MEIJERTOTALECOMTA",
    "RALEYS": "US_USMWCONFTOTAL_RALEYSTOTALCORPECOMTA",
    "RITEAID": "US_USMWCONFTOTAL_RITEAIDTOTALECOMTA",
    "SHOPRITE": "US_USMWCONFTOTAL_SHOPRITETOTALECOMTA",
    "SMARTFINAL": "US_USMWCONFTOTAL_SMARTFINALTOTALECOMTA",
    "TARGET": "US_USMWCONFTOTAL_TARGETTOTALECOMTA",
}
HARRISTEETER_UNIQUE_COL = ["sheetname", "upc"]

In [0]:
spark = SparkSession.builder.appName("spark").getOrCreate()
mars_calendar = spark.table(MARS_CALENDAR)

dim_calendar_date = (
    spark.table(DIM_CALENDAR_DATE)
    .filter("row_latest_ind='Y'")
    .filter("calendar_type='Financial'")
    .select("period_id", "calendar_date")
    .withColumn("calendar_date", F.to_date("calendar_date", "yyyy-MM-dd"))
    .withColumn("calendar_date2", F.regexp_replace(F.col("calendar_date"), "-", ""))
    .withColumn("calendar_date3", F.date_format(F.col("calendar_date"), "yyyyMM"))
)

dim_calendar_period = (
    spark.table(DIM_CALENDAR_PERIOD)
    .filter("row_latest_ind='Y'")
    .filter("calendar_period_type='Financial'")
    .select("calendar_period_id", "period_name")
    .withColumnRenamed("calendar_period_id", "period_id")
)

nielsen_product = (
    spark.table(NIELSEN_PRODUCT)
    .filter(F.col("global_manufacturer") == "MARS WRIGLEY")
    .select("product_id")
)

In [0]:
data = spark.table("dcom_gsc_silver.us_harristeeter_sales")
retailer = "HARRISTEETER"
year_value = "2025"
period_value = "01"
market = "United States"

### HAS_SCHEMA

- Update expected_columns according to your retailer.
- Run the cell and check cell output.

In [0]:
success = 1
expected_columns = {
    "upc": "String",
    "description": "String",
    "sz": "String",
    "uom": "String",
    "subcategory": "String",
    "TOTAL_SALES": "Double",
    "TOTAL_UNITS": "Double",
    "sheetname": "String",
    "filename": "String",
    "year": "String",
    "month": "String"
}

for column, expected_type in expected_columns.items():
    missing_columns = ", ".join(
        [col for col in expected_columns if col not in data.columns]
    )
    schema_success = len(missing_columns) == 0
    actual_type = str(data.schema[column].dataType)
    if schema_success:
        if actual_type.lower() != expected_type.lower():
            success = 0
    print(f'{{"YEAR": "{year_value}", "PERIOD": "{period_value}", "RETAILER": "{retailer}", "OTHER":"{column}"}}')
    print(f"expected_type: {expected_type}, actual_type: {actual_type.split('Type()')[0]}")

{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"upc"}
expected_type: String, actual_type: String
{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"description"}
expected_type: String, actual_type: String
{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"sz"}
expected_type: String, actual_type: String
{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"uom"}
expected_type: String, actual_type: String
{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"subcategory"}
expected_type: String, actual_type: String
{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"TOTAL_SALES"}
expected_type: Double, actual_type: Double
{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"TOTAL_UNITS"}
expected_type: Double, actual_type: Double
{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"sheetname"}
expected_type: String, actual_type: String
{"YEAR": "2

### IS NULL

- Update the values(col names) in cell 12 
- Run the cell 10, 11, 12 and check cell output.

In [0]:
# imports
from pyspark.sql import DataFrame, Window, dataframe
from pyspark.sql.functions import (
    explode,
    create_map,
    split,
    make_date,
    weekofyear,
    to_json,
    struct,
    current_timestamp,
    round,
    substring,
    avg,
    row_number,
    sum,
    col,
    lit,
    regexp_replace,
    when,
    first,
)

In [0]:
def deduplicate_dataframe(df: DataFrame, unique_cols, order_by_col):
    """
    Removes duplicate rows from a DataFrame based on specified unique columns,
    keeping only the most recent row as determined by an ordering column.

    Parameters:
    df (DataFrame): The input Spark DataFrame.
    unique_cols (list or str): Column name(s) to identify duplicates.
    order_by_col (str): Column used to determine the most recent row (e.g., timestamp or ID).

    Returns:
    DataFrame or None: A deduplicated DataFrame with only the top-ranked rows retained 
    for each group of duplicates, or None if the input DataFrame is empty or None.
    """
    if df:
        window_spec = Window.partitionBy(*unique_cols).orderBy(col(order_by_col).desc())
        ranked_df = df.withColumn("rank", row_number().over(window_spec))
        deduplicated_df = ranked_df.filter(col("rank") == 1).drop("rank")
        return deduplicated_df
    else:
        return None


def retailer_null_check(retailer, data, year_col, period_col, year_value, period_value):
    """
    Filters the data for a specific retailer and time period, and checks for non-null records.

    Parameters:
    retailer (str): The retailer name (e.g., 'us_cpl', 'HARRISTEETER').
    data (DataFrame): The input Spark DataFrame.
    year_col (str): The name of the column representing the year (used for 'us_cpl').
    period_col (str): The name of the column representing the period (e.g., month or quarter).
    year_value (int or str): The year value to filter by.
    period_value (int or str): The period value to filter by.

    Returns:
    tuple: A tuple containing:
        - success (bool): True if at least one record exists after filtering, else False.
        - data_size (int): The number of records matching the filter criteria.
        - filtered_data (DataFrame): The filtered Spark DataFrame.
    """
    success = False
    data_size = 0

    if retailer in ["us_cpl"]:
        filtered_df = data.filter(
            (data[year_col] == year_value) & (data[period_col] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer in ["HARRISTEETER"]:
        filtered_df = data.filter(
            (data["year"] == year_value) & (data["month"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer in ["WALMART", "SAMSCLUB"]:
        data_ = data.withColumn(
            "period", F.lpad(F.regexp_replace(F.col("period"), "P", ""), 2, "0")
        )
        filtered_df = data_.filter(
            (data_["year"] == year_value) & (data_["period"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "HEB":
        result_df = dim_calendar_date.join(
            data, dim_calendar_date.calendar_date == data.sales_week_beg_date, "inner"
        )
        result_df2 = result_df.join(
            dim_calendar_period,
            result_df.period_id == dim_calendar_period.period_id,
            "inner",
        )
        result_df3 = result_df2.groupBy("period_name").agg(
            F.countDistinct("sales_week_beg_date").alias("distinct_count")
        )
        result_df4 = result_df3.withColumn(
            "year_value", F.split(F.col("period_name"), "P")[0]
        ).withColumn("period_value", F.split(F.col("period_name"), "P")[1])
        filtered_df = result_df4.filter(
            (F.col("year_value") == year_value)
            & (F.col("period_value") == period_value)
        )
        try:
            if (
                filtered_df.count() > 0
                and filtered_df.collect()[0]["distinct_count"] == 4
            ):
                success = True
            else:
                success = False
        except:
            success = False
        data_size = filtered_df.count()
        filtered_data = filtered_df

    elif retailer in ["STAPLES", "OFFICEDEPOT"]:  # Period
        filtered = data.withColumn("Period", F.concat(F.lit("20"), data["Period"]))
        filtered = filtered.withColumn(
            "year_col", F.substring(F.col("Period"), 1, 4)
        ).withColumn("period_col", F.substring(F.col("Period"), 5, 2))
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered["period_col"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "KROGER_COMPETITOR":  # Period
        filtered = data.withColumn(
            "year_col", F.substring(F.col("Period"), 4, 7)
        ).withColumn("period_col", F.substring(F.col("Period"), 1, 3))
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered[period_col] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer in [
        "AHOLD",
        "ALBSCO",
        "GIANTEAGLE",
        "HYVEE",
        "LOWESFOOD",
        "MEIJER",
        "RALEYS",
        "RITEAID",
        "SHOPRITE",
        "SMARTFINAL",
        "TARGET",
    ]:
        filtered = data.select(
            "product_id", "nielsen_calendar_id", "customer_id", "volume"
        ).withColumn("calendar_date2", F.split(F.col("nielsen_calendar_id"), "_")[2])
        cust = CUSTOMER_DIC[retailer]
        nielsen = (
            filtered.join(dim_calendar_date, "calendar_date2", "left")
            .join(dim_calendar_period, "period_id", "left")
            .withColumn("year", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn("period", F.split(dim_calendar_period["period_name"], "P")[1])
        ).filter(filtered.customer_id == f"{cust}")

        filtered_df = nielsen.filter(
            (nielsen["year"] == year_value) & (nielsen["period"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size == 4

    elif retailer in ["COSTCO", "WALGREENS"]:
        temp_df = data.filter(F.col("retailerid").isin([retailer])).withColumnRenamed(
            "dateid", "calendar_date2"
        )
        filtered = (
            temp_df.join(dim_calendar_date, "calendar_date2", "left")
            .join(dim_calendar_period, "period_id", "left")
            .withColumn("year_col", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn(
                "period_col",
                F.split(dim_calendar_period["period_name"], "P")[1],
            )
        )
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered["period_col"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size == 4

    elif retailer in ["BJS"]:
        df_req_1 = data.withColumnRenamed("dateid", "date_id").withColumnRenamed(
            "retailerid", "customer_name"
        )
        DDAS_ = df_req_1.join(
            dim_calendar_date,
            df_req_1.date_id == dim_calendar_date.calendar_date2,
            "inner",
        )

        DDAS_1 = (
            (
                DDAS_.join(
                    dim_calendar_period,
                    dim_calendar_period["period_id"] == DDAS_["period_id"],
                    how="left",
                )
            )
            .withColumn("year", F.col("period_name").substr(1, 4))
            .withColumn("period", F.col("period_name").substr(6, 2))
        ).filter(F.col("customer_name") == f"{retailer}")

        filtered_df = DDAS_1.filter(
            (DDAS_1["year"] == year_value) & (DDAS_1["period"] == period_value)
        )

        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer in ["CVS", "DOLLARGENERAL"]:
        temp_df = data.filter(F.col("retailerid").isin([retailer])).withColumnRenamed(
            "dateid", "calendar_date2"
        )
        filtered = (
            temp_df.join(dim_calendar_date, "calendar_date2", "left")
            .join(dim_calendar_period, "period_id", "left")
            .withColumn("year_col", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn(
                "period_col",
                F.split(dim_calendar_period["period_name"], "P")[1],
            )
        )
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered["period_col"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 28

    elif retailer in ["RICHARDS", "MASON"]:
        df_US_DDAAS_2 = data.filter(F.col("RetailerID").isin(f"{retailer}"))
        temp = df_US_DDAAS_2.withColumn(
            "Year", F.substring(F.col("dateid"), 1, 4)
        ).withColumn("Month", F.substring(F.col("dateid"), 5, 2))
        delta_df = (
            temp.withColumn(
                "DateID1",
                (
                    F.concat(
                        "Year", F.lit("-"), "Month", F.lit("-"), F.lit(1), F.lit(2)
                    )
                ).cast("date"),
            )
            .withColumn("DateID", F.last_day(F.col("DateID1")))
            .drop("DateID1")
        )

        df = delta_df.select("Year", "Month", "DateID").dropDuplicates()
        df = df.withColumn("Days", F.substring(F.col("DateID"), -2, 2))
        df = df.withColumn("Days", F.col("Days").cast("integer")).withColumn(
            "Month", F.col("Month").cast("integer")
        )
        final_df = df.join(
            mars_calendar,
            (df.Year == mars_calendar.YEAR) & (df.Month == mars_calendar.MONTH_NUMBER),
            how="inner",
        ).select(df["*"], mars_calendar.MARSYEAR, mars_calendar.MARSPERIOD)
        final_df = final_df.withColumn("MARSYEAR", F.col("MARSYEAR").cast("string"))
        filtered_df = final_df.filter(F.col("MARSYEAR") == year_value).filter(
            F.col("MARSPERIOD") == int(period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size == 28

    elif retailer == "AODcomm":
        filtered_df = data.filter(
            (data["year"] == year_value) & (data["period"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "KROGER":
        filtered = (
            data.join(
                dim_calendar_date,
                on=data["Period"] == dim_calendar_date["calendar_date"],
                how="left",
            )
            .join(
                dim_calendar_period,
                on=dim_calendar_date["period_id"] == dim_calendar_period["period_id"],
                how="left",
            )
            .withColumn("year_col", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn(
                "period_col", F.split(dim_calendar_period["period_name"], "P")[1]
            )
        )
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered["period_col"] == period_value)
        )
        filtered_df = filtered_df.select("calendar_date").distinct()
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "MCDONALDS":
        filtered_df = data.withColumn(
            "year_col", F.substring(F.col("Date"), 1, 4)
        ).withColumn("period_col", F.substring(F.col("Date"), 5, 2))
        filtered_df = filtered_df.filter(
            (filtered_df["year_col"] == year_value)
            & (filtered_df["period_col"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "SONIC":
        filtered_df = data.filter(
            (F.col("INVOICEYEAR") == int(year_value))
            & (F.col("MARSPERIOD") == int(period_value))  # Convert period_value to int
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "AMAZON":
        result_df = dim_calendar_date.join(
            data, dim_calendar_date.calendar_date == data.start_date, "inner"
        )
        result_df2 = result_df.join(
            dim_calendar_period,
            result_df.period_id == dim_calendar_period.period_id,
            "inner",
        )

        # Filter the DataFrame for the given year and period
        filtered_df = (
            result_df2.withColumn(
                "year_value", F.split(F.col("period_name"), "P")[0]
            ).withColumn("period_value", F.split(F.col("period_name"), "P")[1])
        ).filter(
            (F.col("year_value") == year_value)
            & (F.col("period_value") == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
    return success, data_size, filtered_data


upc
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "upc"}', 'STATUS': True}
description
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "description"}', 'STATUS': True}
sz
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "sz"}', 'STATUS': True}
uom
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "uom"}', 'STATUS': True}
subcategory
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "subcategory"}', 'STATUS': True}
total_sales
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "total_sales"}', 'STATUS': True}
total_units
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "total_units"}', 'STATUS': True}
sheetname
{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER": "sheetname"}', 'STATUS': True}
filename
{'MESSAGE': '{"YEAR":

In [0]:
success, data_size, filtered_data = retailer_null_check(
    retailer, data, "", "", year_value, period_value
)

values = [
    "upc",
    "description",
    "sz",
    "uom",
    "subcategory",
    "TOTAL_SALES",
    "TOTAL_UNITS",
    "sheetname",
    "filename",
    "year",
    "month"
]
data1 = filtered_data.toDF(*[c.lower() for c in filtered_data.columns])
values = [v.lower() for v in values]

for column in values:
    if column in data1.columns:
        print(column)
        null_count = data1.filter(F.col(column).isNull()).count()
        if null_count > 0:
            result = {
                "MESSAGE": f'{{"YEAR": "{year_value}", "PERIOD": "{period_value}", "RETAILER": "{retailer}", "OTHER": "{column}"}}',
                "STATUS": False
            }
        else:
            result = {
                "MESSAGE": f'{{"YEAR": "{year_value}", "PERIOD": "{period_value}", "RETAILER": "{retailer}", "OTHER": "{column}"}}',
                "STATUS": True
            }
        print(result)


### IS NEG

- Update the values(col names) in cell 15. 
- Run the cell 10, 11, 15 and check cell output.

In [0]:
success, data_size, filtered_data = retailer_null_check(
    retailer, data, "", "", year_value, period_value
)

values = [
"upc"
    ]

data1 = filtered_data.toDF(*[c.lower() for c in filtered_data.columns])
values = [v.lower() for v in values]

for column in values:
    if column in data1.columns:
        negative_count = data1.filter(F.col(column) < 0).count()
        if negative_count > 0:
            result = {
                "MESSAGE": f'{{"YEAR": "{year_value}", "PERIOD": "{period_value}", "RETAILER": "{retailer}", "OTHER":"{column}"}}',
                "STATUS": False
            }
        else:
            result = {
                "MESSAGE": f'{{"YEAR": "{year_value}", "PERIOD": "{period_value}", "RETAILER": "{retailer}", "OTHER":"{column}"}}',
                "STATUS": True
            }
        print(result)

{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER", "OTHER":"upc"}', 'STATUS': True}


# avail_check

- Run the cell 18, 19 and check cell output.

In [0]:

def retailer_avail_check(
    retailer, data, year_col, period_col, year_value, period_value
):
    """
    Checks data availability for a specific retailer and reporting period by applying retailer-specific 
    filtering and transformations.

    This function handles multiple retailers, each potentially requiring custom logic to extract
    relevant records based on a reporting year and period (e.g., month or fiscal period).

    Parameters:
    retailer (str): Name of the retailer (e.g., 'us_cpl', 'HARRISTEETER', 'WALMART', etc.).
    data (DataFrame): Input Spark DataFrame containing transactional or sales data.
    year_col (str): Name of the year column for general filtering (used for some retailers).
    period_col (str): Name of the period column for general filtering (used for some retailers).
    year_value (str or int): Year value to filter on.
    period_value (str or int): Period value to filter on.

    Returns:
    tuple: A tuple containing:
        - success (bool): True if data is considered available for the specified retailer and period,
                          based on logic (e.g., row count, specific count thresholds).
        - data_size (int): The count of relevant records found after filtering.
        - filtered_data (DataFrame): Filtered Spark DataFrame based on retailer-specific logic.

    Notes:
    - Each retailer has custom filtering logic based on how its data is structured.
    - Some retailers require joining with external lookup tables like `dim_calendar_date`, `dim_calendar_period`,
      or `mars_calendar`.
    - Success criteria (e.g., expected row count or date count) vary by retailer:
        - Most use `>= 1`, but some (e.g., CVS, DOLLARGENERAL) expect 28 days of data.
        - Others, like GIANTEAGLE, expect exactly 4 date records.
    - Retailer identifiers must match expected cases (e.g., "us_cpl", "HARRISTEETER", "BJS", "CVS").

    Dependencies:
    - `dim_calendar_date`, `dim_calendar_period`, `mars_calendar` must be accessible in scope.
    - `deduplicate_dataframe`, `CUSTOMER_DIC`, and relevant column formats are assumed to be pre-defined.
    """
    success = False
    data_size = 0

    if retailer in ["us_cpl"]:
        filtered_df = data.filter(
            (data[year_col] == year_value) & (data[period_col] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer in ["HARRISTEETER"]:
        harris_sales_df = deduplicate_dataframe(
            data,
            HARRISTEETER_UNIQUE_COL,
            "insert_date",
        )
        delta_df = (
            harris_sales_df.withColumn(
                "DateID1",
                (
                    F.concat(
                        "Year", F.lit("-"), "Month", F.lit("-"), F.lit(1), F.lit(2)
                    )
                ).cast("date"),
            )
            .withColumn("DateID", F.last_day(F.col("DateID1")))
            .drop("DateID1")
        )

        df = delta_df.select("Year", "Month", "DateID").dropDuplicates()
        df = df.withColumn("Days", F.substring(F.col("DateID"), -2, 2))
        df = df.withColumn("Days", F.col("Days").cast("integer")).withColumn(
            "Month", F.col("Month").cast("integer")
        )
        final_df = df.join(
            mars_calendar,
            (df.Year == mars_calendar.YEAR) & (df.Month == mars_calendar.MONTH_NUMBER),
            how="inner",
        ).select(df["*"], mars_calendar.MARSYEAR, mars_calendar.MARSPERIOD)
        final_df = final_df.withColumn("MARSYEAR", F.col("MARSYEAR").cast("string"))
        filtered_df = (
            final_df.filter(F.col("MARSYEAR") == year_value)
            .filter(F.col("MARSPERIOD") == int(period_value))
            .groupBy("MARSPERIOD", "MARSYEAR")
            .agg(F.count("MARSPERIOD").alias("count"))
        )
        try:
            count_value = filtered_df.select("count").collect()[0][0]
        except:
            count_value = 0
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = count_value >= 1

    elif retailer in ["WALMART", "SAMSCLUB"]:
        data_ = data.withColumn(
            "period", F.lpad(F.regexp_replace(F.col("period"), "P", ""), 2, "0")
        )
        filtered_df = data_.filter(
            (data_["year"] == year_value) & (data_["period"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "HEB":
        result_df = dim_calendar_date.join(
            data, dim_calendar_date.calendar_date == data.sales_week_beg_date, "inner"
        )
        result_df2 = result_df.join(
            dim_calendar_period,
            result_df.period_id == dim_calendar_period.period_id,
            "inner",
        )
        result_df3 = result_df2.groupBy("period_name").agg(
            F.countDistinct("sales_week_beg_date").alias("distinct_count")
        )
        result_df4 = result_df3.withColumn(
            "year_value", F.split(F.col("period_name"), "P")[0]
        ).withColumn("period_value", F.split(F.col("period_name"), "P")[1])
        filtered_df = result_df4.filter(
            (F.col("year_value") == year_value)
            & (F.col("period_value") == period_value)
        )
        try:
            if (
                filtered_df.count() > 0
                and filtered_df.collect()[0]["distinct_count"] == 4
            ):
                success = True
            else:
                success = False
        except:
            success = False
        data_size = filtered_df.count()
        filtered_data = filtered_df

    elif retailer in ["STAPLES", "OFFICEDEPOT"]:  # Period
        filtered = data.withColumn("Period", F.concat(F.lit("20"), data["Period"]))
        filtered = filtered.withColumn(
            "year_col", F.substring(F.col("Period"), 1, 4)
        ).withColumn("period_col", F.substring(F.col("Period"), 5, 2))
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered["period_col"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "KROGER_COMPETITOR":  # Period
        filtered = data.withColumn(
            "year_col", F.substring(F.col("Period"), 4, 7)
        ).withColumn("period_col", F.substring(F.col("Period"), 1, 3))
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered[period_col] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer in [
        "AHOLD",
        "ALBSCO",
        "GIANTEAGLE",
        "HYVEE",
        "LOWESFOOD",
        "MEIJER",
        "RALEYS",
        "RITEAID",
        "SHOPRITE",
        "SMARTFINAL",
        "TARGET",
    ]:
        filtered = data.select(
            "product_id", "nielsen_calendar_id", "customer_id", "volume"
        ).withColumn("calendar_date2", F.split(F.col("nielsen_calendar_id"), "_")[2])
        cust = CUSTOMER_DIC[retailer]
        nielsen = (
            filtered.join(dim_calendar_date, "calendar_date2", "left")
            .join(dim_calendar_period, "period_id", "left")
            .withColumn("year", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn("period", F.split(dim_calendar_period["period_name"], "P")[1])
        ).filter(filtered.customer_id == f"{cust}")

        filtered_df = (
            nielsen.filter(
                (nielsen["year"] == year_value) & (nielsen["period"] == period_value)
            )
            .groupBy("year", "period", "calendar_date")
            .agg(F.count("calendar_date").alias("date_count"))
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size == 4

    elif retailer in ["COSTCO", "WALGREENS"]:
        temp_df = data.filter(F.col("retailerid").isin([retailer])).withColumnRenamed(
            "dateid", "calendar_date2"
        )
        filtered = (
            temp_df.join(dim_calendar_date, "calendar_date2", "left")
            .join(dim_calendar_period, "period_id", "left")
            .withColumn("year_col", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn(
                "period_col",
                F.split(dim_calendar_period["period_name"], "P")[1],
            )
        )
        filtered_df = (
            filtered.filter(
                (filtered["year_col"] == year_value)
                & (filtered["period_col"] == period_value)
            )
            .groupBy("year_col", "period_col", "calendar_date")
            .agg(F.count("calendar_date").alias("date_count"))
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size == 4

    elif retailer in ["BJS"]:
        df_req_1 = data.withColumnRenamed("DateID", "Date_ID").withColumnRenamed(
            "RetailerID", "customer_name"
        )
        DDAS_ = df_req_1.join(
            dim_calendar_date,
            df_req_1.Date_ID == dim_calendar_date.calendar_date2,
            "inner",
        )

        DDAS_1 = (
            (
                DDAS_.join(
                    dim_calendar_period,
                    dim_calendar_period["period_id"] == DDAS_["period_id"],
                    how="left",
                )
            )
            .withColumn("year", F.col("period_name").substr(1, 4))
            .withColumn("period", F.col("period_name").substr(6, 2))
        ).filter(F.col("customer_name") == f"{retailer}")

        filtered_df = (
            DDAS_1.filter(
                (DDAS_1["year"] == year_value) & (DDAS_1["period"] == period_value)
            )
            .groupBy("year", "period", "customer_name")
            .agg(F.count("customer_name").alias("customer_name_count"))
        )

        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer in ["CVS", "DOLLARGENERAL"]:
        temp_df = data.filter(F.col("retailerid").isin([retailer])).withColumnRenamed(
            "dateid", "calendar_date2"
        )
        filtered = (
            temp_df.join(dim_calendar_date, "calendar_date2", "left")
            .join(dim_calendar_period, "period_id", "left")
            .withColumn("year_col", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn(
                "period_col",
                F.split(dim_calendar_period["period_name"], "P")[1],
            )
        )
        filtered_df = (
            filtered.filter(
                (filtered["year_col"] == year_value)
                & (filtered["period_col"] == period_value)
            )
            .groupBy("year_col", "period_col", "calendar_date")
            .agg(F.count("calendar_date").alias("date_count"))
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 28

    elif retailer in ["RICHARDS", "MASON"]:
        df_US_DDAAS_2 = data.filter(F.col("RetailerID").isin(f"{retailer}"))
        temp = df_US_DDAAS_2.withColumn(
            "Year", F.substring(F.col("dateid"), 1, 4)
        ).withColumn("Month", F.substring(F.col("dateid"), 5, 2))
        delta_df = (
            temp.withColumn(
                "DateID1",
                (
                    F.concat(
                        "Year", F.lit("-"), "Month", F.lit("-"), F.lit(1), F.lit(2)
                    )
                ).cast("date"),
            )
            .withColumn("DateID", F.last_day(F.col("DateID1")))
            .drop("DateID1")
        )

        df = delta_df.select("Year", "Month", "DateID").dropDuplicates()
        df = df.withColumn("Days", F.substring(F.col("DateID"), -2, 2))
        df = df.withColumn("Days", F.col("Days").cast("integer")).withColumn(
            "Month", F.col("Month").cast("integer")
        )
        final_df = df.join(
            mars_calendar,
            (df.Year == mars_calendar.YEAR) & (df.Month == mars_calendar.MONTH_NUMBER),
            how="inner",
        ).select(df["*"], mars_calendar.MARSYEAR, mars_calendar.MARSPERIOD)
        final_df = final_df.withColumn("MARSYEAR", F.col("MARSYEAR").cast("string"))
        filtered_df = (
            final_df.filter(F.col("MARSYEAR") == year_value)
            .filter(F.col("MARSPERIOD") == int(period_value))
            .groupBy("MARSPERIOD", "MARSYEAR")
            .agg(F.count("MARSPERIOD").alias("count"))
        )
        try:
            data_size = filtered_df.select("count").collect()[0][0]
        except:
            data_size = 0
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size == 28

    elif retailer == "AODcomm":
        filtered_df = data.filter(
            (data["year"] == year_value) & (data["period"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "KROGER":
        filtered = (
            data.join(
                dim_calendar_date,
                on=data["Period"] == dim_calendar_date["calendar_date"],
                how="left",
            )
            .join(
                dim_calendar_period,
                on=dim_calendar_date["period_id"] == dim_calendar_period["period_id"],
                how="left",
            )
            .withColumn("year_col", F.split(dim_calendar_period["period_name"], "P")[0])
            .withColumn(
                "period_col", F.split(dim_calendar_period["period_name"], "P")[1]
            )
        )
        filtered_df = filtered.filter(
            (filtered["year_col"] == year_value)
            & (filtered["period_col"] == period_value)
        )
        filtered_df = filtered_df.select("calendar_date").distinct()
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "MCDONALDS":
        filtered_df = data.withColumn(
            "year_col", F.substring(F.col("Date"), 1, 4)
        ).withColumn("period_col", F.substring(F.col("Date"), 5, 2))
        filtered_df = filtered_df.filter(
            (filtered_df["year_col"] == year_value)
            & (filtered_df["period_col"] == period_value)
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "SONIC":
        filtered_df = data.filter(
            (F.col("INVOICEYEAR") == int(year_value))
            & (F.col("MARSPERIOD") == int(period_value))  # Convert period_value to int
        )
        data_size = filtered_df.count()
        filtered_data = filtered_df
        success = data_size >= 1

    elif retailer == "AMAZON":
        result_df = dim_calendar_date.join(
            data, dim_calendar_date.calendar_date == data.start_date, "inner"
        )
        result_df2 = result_df.join(
            dim_calendar_period,
            result_df.period_id == dim_calendar_period.period_id,
            "inner",
        )
        result_df3 = result_df2.groupBy("period_name").agg(
            F.countDistinct("start_date").alias("distinct_count")
        )

        result_df4 = result_df3.withColumn(
            "year_value", F.split(F.col("period_name"), "P")[0]
        ).withColumn("period_value", F.split(F.col("period_name"), "P")[1])

        # Filter the DataFrame for the given year and period
        filtered_df = result_df4.filter(
            (F.col("year_value") == year_value)
            & (F.col("period_value") == period_value)
        )

        try:
            if (
                filtered_df.count() > 0
                and filtered_df.collect()[0]["distinct_count"] == 28
            ):
                success = True
            else:
                success = False
        except:
            success = False
        data_size = filtered_df.count()
        filtered_data = filtered_df
    return success, data_size, filtered_data


{'MESSAGE': '{"YEAR": "2025", "PERIOD": "01", "RETAILER": "HARRISTEETER"}', 'STATUS': True}


In [0]:
success, data_size, filtered_data = retailer_avail_check(
    retailer, data, year_value, period_value, year_value, period_value
)

result = {
    "MESSAGE": f'{{"YEAR": "{year_value}", "PERIOD": "{period_value}", "RETAILER": "{retailer}"}}',
    "STATUS": success
}
print(result)

# DQ Table
- To check the DQ records

In [0]:
table = spark.table("dcom_gsc_dq.dq_results")
filtered = table.filter(table["PROCESS_NAME"] == "HARRISTEETER")\
    .filter(table["PROCESS_DATE"] == "20250418")
display(filtered)

DATAFRAME_NAME,TEST_NAME,MESSAGE,APP_ID,STATUS,DATASET,VALUE,TYPE_OF_DATASET,DQ_CHECK_TIMESTAMP,PROJECT_NAME,PROCESS_NAME,PROCESS_DATE
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""02"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""filename""}",app-20250418043735-0000,Success,filename,"expected_type: string, actual_type: String",Column,2025-04-18T04:54:31.552Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""02"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""year""}",app-20250418043735-0000,Success,year,"expected_type: string, actual_type: String",Column,2025-04-18T04:54:31.552Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""02"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""month""}",app-20250418043735-0000,Success,month,"expected_type: string, actual_type: String",Column,2025-04-18T04:54:31.552Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""02"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""upc""}",app-20250418043735-0000,Success,upc,"expected_type: string, actual_type: String",Column,2025-04-18T04:54:31.552Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""02"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""description""}",app-20250418043735-0000,Success,description,"expected_type: string, actual_type: String",Column,2025-04-18T04:54:31.552Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""01"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""filename""}",app-20250418034156-0000,Success,filename,"expected_type: string, actual_type: String",Column,2025-04-18T04:28:24.173Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""01"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""year""}",app-20250418034156-0000,Success,year,"expected_type: string, actual_type: String",Column,2025-04-18T04:28:24.173Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""01"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""month""}",app-20250418034156-0000,Success,month,"expected_type: string, actual_type: String",Column,2025-04-18T04:28:24.173Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""02"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""sz""}",app-20250418043735-0000,Success,sz,"expected_type: string, actual_type: String",Column,2025-04-18T04:54:31.552Z,ODDA-DCOMGSC,HARRISTEETER,20250418
us_harristeeter_sales,has_schema,"{""YEAR"": ""2025"", ""PERIOD"": ""02"", ""RETAILER"": ""HARRISTEETER"", ""OTHER"":""uom""}",app-20250418043735-0000,Success,uom,"expected_type: string, actual_type: String",Column,2025-04-18T04:54:31.552Z,ODDA-DCOMGSC,HARRISTEETER,20250418


# Data Avail Example Run for AHOLD

In [0]:
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql import functions as F
retailer = "AHOLD"
year_value = "2025"
period_value = "02"
CUSTOMER_DIC = {
    "AHOLD": "US_USMWCONFTOTAL_AHOLDDELHAIZETOTALSTOREDELIVERYTA",
    "ALBSCO": "US_USMWCONFTOTAL_ALBSCOTOTALECOMTA",
    "GIANTEAGLE": "US_USMWCONFTOTAL_GIANTEAGLETOTALECOMTA",
    "HYVEE": "US_USMWCONFTOTAL_HYVEETOTALECOMTA",
    "LOWESFOOD": "US_USMWCONFTOTAL_LOWESFOODTOTALECOMTA",
    "MEIJER": "US_USMWCONFTOTAL_MEIJERTOTALECOMTA",
    "RALEYS": "US_USMWCONFTOTAL_RALEYSTOTALCORPECOMTA",
    "RITEAID": "US_USMWCONFTOTAL_RITEAIDTOTALECOMTA",
    "SHOPRITE": "US_USMWCONFTOTAL_SHOPRITETOTALECOMTA",
    "SMARTFINAL": "US_USMWCONFTOTAL_SMARTFINALTOTALECOMTA",
    "TARGET": "US_USMWCONFTOTAL_TARGETTOTALECOMTA",
}
MARS_CALENDAR = "raw_manual_us.mars_calendar"
DIM_CALENDAR_DATE = "cdm_common.calendar_date"
DIM_CALENDAR_PERIOD = "cdm_common.calendar_period"
NIELSEN_PRODUCT = "dcom_gsc_silver.us_nielsen_product"
spark = SparkSession.builder.appName("spark").getOrCreate()
mars_calendar = spark.table(MARS_CALENDAR)

dim_calendar_date = (
    spark.table(DIM_CALENDAR_DATE)
    .filter("row_latest_ind='Y'")
    .filter("calendar_type='Financial'")
    .select("period_id", "calendar_date")
    .withColumn("calendar_date", F.to_date("calendar_date", "yyyy-MM-dd"))
    .withColumn("calendar_date2", F.regexp_replace(F.col("calendar_date"), "-", ""))
    .withColumn("calendar_date3", F.date_format(F.col("calendar_date"), "yyyyMM"))
)

dim_calendar_period = (
    spark.table(DIM_CALENDAR_PERIOD)
    .filter("row_latest_ind='Y'")
    .filter("calendar_period_type='Financial'")
    .select("calendar_period_id", "period_name")
    .withColumnRenamed("calendar_period_id", "period_id")
)

nielsen_product = (
    spark.table(NIELSEN_PRODUCT)
    .filter(F.col("global_manufacturer") == "MARS WRIGLEY")
    .select("product_id")
)
data = spark.read.format("delta").load("/mnt/oddacentral_prod_gen2/raw/trusted/ddaas/sdm/local_fact_local_data")
filtered = data.select(
    "product_id", "nielsen_calendar_id", "customer_id", "volume"
).withColumn("calendar_date2", F.split(F.col("nielsen_calendar_id"), "_")[2])
cust = CUSTOMER_DIC[retailer]
nielsen = (
    filtered.join(dim_calendar_date, "calendar_date2", "left")
    .join(dim_calendar_period, "period_id", "left")
    .withColumn("year", F.split(dim_calendar_period["period_name"], "P")[0])
    .withColumn("period", F.split(dim_calendar_period["period_name"], "P")[1])
).filter(filtered.customer_id == f"{cust}")

filtered_df = (
    nielsen.filter(
        (nielsen["year"] == year_value) & (nielsen["period"] == period_value)
    )
    .groupBy("year", "period", "calendar_date")
    .agg(F.count("calendar_date").alias("date_count"))
)
data_size = filtered_df.count()
filtered_data = filtered_df
success = data_size == 4
print(success)
display(filtered_data)

True


year,period,calendar_date,date_count
2025,02,2025-02-15,857
2025,02,2025-02-01,892
2025,02,2025-02-22,887
2025,02,2025-02-08,856
